## References
* [GitHub - Data Engineering Helpers - Strava Data (this project)](https://github.com/data-engineering-helpers/databricks-examples)


In [1]:
# Import Libraries
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import SparkSession

# Setup the Configuration
#conf = pyspark.SparkConf()

# Initialize the Spark session through Spark Connect
spark = SparkSession.builder.getOrCreate()

## Authentication with the Strava API
* Reference:
  [Setup/Get a Stava API access token section](https://github.com/data-engineering-helpers/strava-data#get-a-stava-api-access-token)
* Be sure to have the following Strava API details at hand,
  from https://www.strava.com/settings/api:
  + Client ID
  + Client secret

### Generate an authentication token/code with the internet browser
* Get a Strava API authorization code by opening the following link (be careful
  to replace `strava-api-client-id` by your own Strava API client ID,
  which should read something like 1234567):
  + Open http://www.strava.com/oauth/authorize?client_id=strava-api-client-id&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=read_all,activity:read_all
  + Click on the Authorize button
  + In the URL of the just opened page (displaying an error), copy the `code`
    value, it corresponds to the Strava API authorization code

### Generate/refresh the access and refresh tokens with the CLI
* Use cURL on the command-line to create the access code (and the refresh code):
```bash
$ curl -s -X POST https://www.strava.com/oauth/token -F client_id=<strava-api-client-id> -F client_secret=<strava-api-client-secret> -F code=<strava-api-authorization-code> -F grant_type=authorization_code | jq
```

### Store the access token as environment variable
* Store the access token as an environment variable:
```bash
$ export STRAVA_ACCESS_TOKEN="<the-strava-api-access-token>"
```

In [1]:
# Strava API access token, usually valid  hours
strava_access_token: str = "6c15aa908aa467b014973b703c44ca7c81986811"

## Retrieve all the activities of the current user
* The following cell/script retrieves, from the Strava API,
  all the activities/trips for the specific user, who has been
  authenticated by the procedure above.
* The trips are saved in a CSV file, namely
  http://localhost:8889/lab/tree/data/private/strava-activity-polylines.csv

In [2]:
import requests
import csv

headers = {'Authorization': "Bearer {0}".format(strava_access_token)}

with open("../data/private/strava-activity-polylines.csv", "w") as trip_file:
    writer = csv.writer(trip_file, delimiter=",")
    writer.writerow(["id", "polyline"])

    page = 1
    while True:
        r = requests.get("https://www.strava.com/api/v3/athlete/activities?page={0}".format(page), headers = headers)
        response = r.json()

        if len(response) == 0:
            break
        else:
            for activity in response:
                r = requests.get("https://www.strava.com/api/v3/activities/{0}?include_all_efforts=true".format(activity["id"]), headers = headers)
                polyline = r.json()["map"]["polyline"]
                writer.writerow([activity["id"], polyline])
            page += 1


## Parse the CSV file

In [5]:
import csv

trip_list: list = []
with open("../data/private/strava-activity-polylines.csv", "r") as trip_file:
    reader = csv.DictReader(trip_file)

    for row in reader:
        trip_list.append(row["polyline"])

len(trip_list)

28

## Display the trips on Leaflet/Folium

In [8]:
import folium
import polyline
from folium import plugins

m = folium.Map()
for trip in trip_list:
    poly = polyline.decode(trip)
    #print(f"Run: {poly}")
    folium.PolyLine(locations=poly).add_to(m)

# Center the map and zoom so as to see all the activities at once
m.fit_bounds(m.get_bounds())

# Store the map into a static HTML file, so that it may shared without need to rerun everything
m.save("../html/private/strava-activity-detail.html")

m

## Open the map in another browser tab
* Open http://localhost:8889/lab/tree/html/private/strava-activity-detail.html
* Click on the "Trust HTML" button
